<a href="https://colab.research.google.com/github/Arutselvan/ASU-CSE-576-NLP/blob/main/HW_2/com_qa_task_instances.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets

In [ ]:
from datasets import load_dataset, concatenate_datasets
dataset = load_dataset("com_qa")

Using custom data configuration default
Reusing dataset com_qa (/root/.cache/huggingface/datasets/com_qa/default/0.1.0/3f5b9b53f3ff9c3423018530911f2031542f6a8b66c6709f69475ab27e43df63)


In [ ]:
merged_dataset = concatenate_datasets([dataset['train'],dataset['test'],dataset['validation']])
len(merged_dataset)

7175

In [ ]:
rephrase_dataset = merged_dataset.filter(lambda x: len(x['questions'])>1)

rephrase_instances = []

for cluster in rephrase_dataset:
  rephrase_instances.append({
      "input": "Question: " + cluster['questions'][0],
      "output": cluster['questions'][1:]
  })

len(rephrase_instances)

Loading cached processed dataset at /root/.cache/huggingface/datasets/com_qa/default/0.1.0/3f5b9b53f3ff9c3423018530911f2031542f6a8b66c6709f69475ab27e43df63/cache-e6244efb342920df.arrow


1805

In [ ]:
wikipedia_dataset = merged_dataset.filter(lambda x: "https://en.wikipedia.org" in x['answers'][0])

topic_questions_instances = []

for wikians in wikipedia_dataset:
  topic_questions_instances.append({
      "input": "Answer: " + wikians['answers'][0].split('/')[-1].replace('_',' '),
      "output": wikians['questions']
  })

len(topic_questions_instances)

Loading cached processed dataset at /root/.cache/huggingface/datasets/com_qa/default/0.1.0/3f5b9b53f3ff9c3423018530911f2031542f6a8b66c6709f69475ab27e43df63/cache-3b959d55eaebf71f.arrow


4982

In [ ]:
wikilink_questions_instances = []

for wikilink in wikipedia_dataset:
  wikilink_questions_instances.append({
      "input": "Questions: " + str(wikilink['questions']),
      "output": [wikilink['answers'][0]]
  })


In [ ]:
answer_dataset = merged_dataset.filter(lambda x: len(x['answers'])==1 and "wikipedia" not in x['answers'][0] )

direct_answer_questions = []

for answer in answer_dataset:
  direct_answer_questions.append({
      "input": "Questions: " + str(answer['questions']),
      "output": [answer['answers'][0]]
  })

direct_answer_questions = [x for x in direct_answer_questions if x['output']!=[""] or x['output']!=['']]

len(direct_answer_questions)


Loading cached processed dataset at /root/.cache/huggingface/datasets/com_qa/default/0.1.0/3f5b9b53f3ff9c3423018530911f2031542f6a8b66c6709f69475ab27e43df63/cache-fe0fb4777c4b6ff7.arrow


1574

In [ ]:
import json

CONTRIBUTOR = "Arut Selvan Dhanasekaran"
SOURCE = "opus_paracrawl"

In [ ]:
# Task 1

definition_1 = "Given a question, create questions that are rephrasals of the original question. Additional contextual information if available can be used to frame the questions. However answer for all the generated questions should be same and should match the answer of the original question."

positive_examples_1 = [{
    "input": "Question: what places in africa do people speak french?",
    "output": '["where in africa do people speak french?", "what countries in africa speak french?", "where in africa do people speak french?"]',
    "explanation": "The generated questions mean the same as the input question and the answer will be the same for all the questions"
},
{
    "input": "Question: hitler became chancellor of germany in what year?",
    "output": '["what year did hitler become chancellor of germany?", "what year was hitler elected the chancellor of germany?", "in which year did hitler become the chancellor of germany?", "what year did adolf hitler become chancellor for germany?", "which date did hitler become chancellor of germany?"]',
    "explanation": "The answer for all the generated questions and the given question can be given by a single date"
},
{
    "input": "Question: mandeville lousiana zip code?",
    "output": '["what is the zip code for mandeville louisiana?", "what is mandeville louisiana 9 digit zip code?"]',
    "explanation": "The rephrased questions exactly means the same as the given question even though some additional contextual information was used in one of the questions created."
}]

negative_examples_1 = [{
    "input": "Question: How long is the Nile river?",
    "output": '["Is Nile the longest river?"]',
    "explanation": "The question created changes the meaning of the input question and the answer is not the same for the questions"
},
{
    "input": "Question: Who is the first football player to win seven balon d'ors?",
    "output": '["Who won the first balon dor?", "who is the first football player?"]',
    "explanation": "The generated questions don't mean the same as the original questions and the answers are different for each question."
},
{
    "input": "Question: How Vitamin C helps the human body?",
    "output": '["How vitamins help humans?", "Does Vitamin C help the human body?"]',
    "explanation": "One of the questions created generalises a specific question. The other changes the meaning of the given question."
}]

temp_set = set()
rephrase_instances_cleaned = []
for instance in rephrase_instances:
  if instance['input'] not in temp_set:
    temp_set.add(instance['input'])
    rephrase_instances_cleaned.append(instance)

  

task_1 = {
    "Contributors": [CONTRIBUTOR],
    "Source": [SOURCE],
    "Categories": ["Question Generation"],
    "Definition": definition_1,
    "Positive Examples": positive_examples_1,
    "Negative Examples": negative_examples_1,
    "Instances": rephrase_instances_cleaned
}

with open("task145_com_qa_question_generation.json", 'w') as fout:
    json_dumps_str = json.dumps(task_1, indent=4)
    print(json_dumps_str, file=fout)



In [ ]:
# Task 2

definition_2 = "An answer is given. Generate questions directly related to that answer. Don't create questions on topics indirectly related to the given answer. The questions generated should be directly or partially answered by the given answer."

positive_examples_2 = [{
    "input": "Answer: victoria woodhull",
    "output": '["who was the first women to run for presidency in the us?", "who was the first women to run for the us president?", "which women was the first to run for president in the us?", "first women to run for presidency of the usa?"]',
    "explanation": "The given answer directly answers all the generated questions"
},
{
    "input": "Answer: atlanta",
    "output": '["Top 3 largest citites in Georgia?"]',
    "explanation": "Although the answer doesn't directly answer the question, the given answer is a partial answer to the generated question and relates directly to it."
},
{
    "input": "Answer: united states presidential election, 1860",
    "output": '["when was abraham lincoln voted for president?", "when was president lincoln voted president?", "when was lincoln president?"]',
    "explanation": "The questions generated can be answered by the given answer"
}]

negative_examples_2 = [{
    "input": "Answer: Infrared rays",
    "output": '["X rays are in which part of the electromagnetic spectrum?"]',
    "explanation": "The question and the answer is a part of a broader topic but they're not directly related"
},
{
    "input": "Answer: Coca Cola",
    "output": '["Are soft drinks harmful to health?", "Is Pepsi better?"]',
    "explanation": "The questions are indirectly related to the given answer but in no way the given answer directly or indirectly answers the generated questions"
},
{
    "input": "Answer: Martin Luther King",
    "output": '["Who is the first African-American president of USA?", "How prevalent was racial inequality in the 2000s?"]',
    "explanation": "The answer and questions are not directly related and the answer doesn't even indirectly answer the questions generated"
}]

temp_set = set()
topic_questions_instances_cleaned = []
for instance in topic_questions_instances:
  if instance['input'] not in temp_set:
    temp_set.add(instance['input'])
    topic_questions_instances_cleaned.append(instance)

task_2 = {
    "Contributors": [CONTRIBUTOR],
    "Source": [SOURCE],
    "Categories": ["Question Generation"],
    "Definition": definition_2,
    "Positive Examples": positive_examples_2,
    "Negative Examples": negative_examples_2,
    "Instances": topic_questions_instances_cleaned
}

with open("task146_com_qa_question_generation.json", 'w') as fout:
    json_dumps_str = json.dumps(task_2, indent=4)
    print(json_dumps_str, file=fout)

In [ ]:
# Task 3

definition_3 = "Given a group of questions that mean the same thing, answer the question. The answer should exactly answer all the questions given without any ambiguity. Don't give related answers or partial answers."

positive_examples_3 = [{
    "input": "Questions: ['what is the amount of us representatives of nevada?', 'what are the us representative number for nevada?','what is the number of us representatives in nevada?']",
    "output": "4",
    "explanation": "The output answer is the exact answer for the given questions"
},
{
    "input": "Questions: ['when did bill clinton and hillary clinton married?', 'when did bill clinton get married to hillary rodham clinton?', 'what year did bill clinton marry hillary rodham clinton?', 'when did bill clinton and hillary clinton get married?', 'when did hillary clinton marry bill?', 'what date did bill clinton and hillary clinton get married?']",
    "output": "1975-10-11",
    "explanation": "All the questions are answered by the generated answer without any ambiguity"
},
{
    "input": "Questions: ['what year did apollo 11 first land on the moon?', 'what year did apollo 11's first astronauts land on the moon?', 'what year the apollo 11 arrived for the first time into the moon?']",
    "output": "1969",
    "explanation": "The questions are directly answered by the output answer"
}]

negative_examples_3 = [{
    "input": "Questions:['What is the name of the Formula 1 Circuit in India?', 'Which circuit hosted the first Indian Grand Prix?'] ",
    "output": "Delhi",
    "explanation": "The output answer doesn't correctly answer the give questions even though the exact answer is related to the output answer."
},
{
    "input": "Questions: ['What pigment is used by leaves for generating energy?', 'What gives leaves it's green color?']",
    "output": "Photosynthesis",
    "explanation": "The questions are not directly answered by the answer"
},
{
    "input": "Questions: ['What is the lastest version of macOS called?']",
    "output": "Apple",
    "explanation": "The question and the answer are indrectly related but is not an exact answer to the given question"
}]

temp_set = set()
daq_cleaned = []
for instance in direct_answer_questions:
  if instance['input'] not in temp_set:
    temp_set.add(instance['input'])
    daq_cleaned.append(instance)

task_3 = {
    "Contributors": [CONTRIBUTOR],
    "Source": [SOURCE],
    "Categories": ["Answer Generation"],
    "Definition": definition_3,
    "Positive Examples": positive_examples_3,
    "Negative Examples": negative_examples_3,
    "Instances": daq_cleaned
}

with open("task147_com_qa_answer_generation.json", 'w') as fout:
    json_dumps_str = json.dumps(task_3, indent=4)
    print(json_dumps_str, file=fout)


In [ ]:
# Task 4

definition_4 = "Given a set of questions, give a Wikipedia page link that might answer all the questions. The Wikipedia page should be directly related to the set of questions"

positive_examples_4 = [{
    "input": "Questions: ['which american president held office for the shortest amount of time?', 'which president held office for the shortest period of time?', 'which person in the us held office for the shortest amount of time?', 'which us president ruled for the shortest time?', 'what president of the united states held office for the shortest period?', 'what us president stayed in office for the shortest amount of time?']",
    "output": "https://en.wikipedia.org/wiki/william_henry_harrison",
    "explanation": "The wikipedia page is related to all the questions and is also an direct answer to all the questions."
},
{
    "input": "Questions: ['who was the first female to be on a wheaties box?', 'first female olympic athlete on wheaties box?', 'who was the first female athlete to be on the wheaties box?']",
    "output": "https://en.wikipedia.org/wiki/mary_lou_retton",
    "explanation": "The given link contains information about the exact answer of the all questions."
},
{
    "input": "Questions: ['what is the fourth biggest city in scotland?']",
    "output": "https://en.wikipedia.org/wiki/dundee",
    "explanation": "The linked wikipedia page is directly related to the given question"
}]

negative_examples_4 = [{
    "input": "Questions: ['What is the name of the Internationa airport in Phoenix?', 'Name any one of Arizona's international airports']",
    "output": "https://en.wikipedia.org/wiki/Arizona_State_University",
    "explanation": "The wikipedia page is not directly related to the given questions even though the answer might be present in the page"
},
{
    "input": "Questions: ['How did World War I start?', 'Who started world war 1?']",
    "output": "https://en.wikipedia.org/wiki/World_War_II",
    "explanation": "The given link does not contain any relevant information about the questions asked"
},
{
    "input": "Questions: ['What is the largest city by population?', 'Which city has the highest population?']",
    "output": "https://en.wikipedia.org/wiki/Chongqing",
    "explanation": "The linked wikipedia page is not the correct answer for the given questions."
}]

temp_set = set()
wl_cleaned = []
for instance in wikilink_questions_instances:
  if instance['input'] not in temp_set:
    temp_set.add(instance['input'])
    wl_cleaned.append(instance)

task_4 = {
    "Contributors": [CONTRIBUTOR],
    "Source": [SOURCE],
    "Categories": ["Answer Generation"],
    "Definition": definition_4,
    "Positive Examples": positive_examples_4,
    "Negative Examples": negative_examples_4,
    "Instances": wl_cleaned
}

with open("task148_com_qa_answer_generation.json", 'w') as fout:
    json_dumps_str = json.dumps(task_4, indent=4)
    print(json_dumps_str, file=fout)